In [1]:
%%sh

# The name of our algorithm
algorithm_name=lidar

cd container

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-east-1 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"


# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Get the login command from ECR in order to pull down the SageMaker PyTorch image
$(aws ecr get-login --registry-ids 520713654638 --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} . --build-arg REGION=${region}
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/chase/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded


WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/chase/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded
Sending build context to Docker daemon  13.82kB
Step 1/6 : ARG REGION=us-west-2
Step 2/6 : FROM 520713654638.dkr.ecr.$REGION.amazonaws.com/sagemaker-pytorch:0.4.0-cpu-py3
0.4.0-cpu-py3: Pulling from sagemaker-pytorch
6b524a54a844: Pulling fs layer
65b9175cb1f0: Pulling fs layer
99f3f7db9f39: Pulling fs layer
98f58a51cdc0: Pulling fs layer
e4fcbd76f95b: Pulling fs layer
bab6d6e13f8b: Pulling fs layer
196bb884b873: Pulling fs layer
f9c314f81719: Pulling fs layer
0e7c80dae4bb: Pulling fs layer
98f58a51cdc0: Waiting
e4fcbd76f95b: Waiting
196bb884b873: Waiting
f9c314f81719: Waiting
bab6d6e13f8b: Waiting
6adca18c522e: Pulling fs layer
0e7c80dae4bb: Waiting
64f29009eb35: Pulling fs layer
3fc548ae183b: Pulling fs layer
65b9175cb1f0: Verifying Checksum
65b9175cb1f0: Download complete
99f3f7db9f39: Download complete
98f58a51cdc0: Verifying Checksum
98f58a51cdc0: Download complete
bab6d6e13f8b: Verifying Checksum
bab6d6e13f8b: Download complete
6b524a54a844: Verifying Checksum
6b5